# IKEA MEPNet Adapter Training

This notebook trains the MEPNet model adapted for IKEA furniture assembly instructions.

**Requirements:**
- GPU runtime (T4 or better recommended)
- ~15GB disk space for dataset
- ~8GB GPU memory

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/hpend2373/NLP.git
%cd NLP/ikea_mepnet_adapter

In [ ]:
# Install dependencies
!pip install -q torch torchvision torchaudio
!pip install -q opencv-python pillow scipy scikit-image
!pip install -q trimesh pyyaml tqdm
!pip install -q wandb tensorboard

# Install ChamferDistance (for GPU)
!pip install -q chamferdist

In [ ]:
# Verify PyTorch GPU setup
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Download IKEA Dataset

The IKEA Manuals at Work dataset contains furniture assembly instructions with 3D models and annotations.

In [ ]:
# Download dataset (this will take several minutes)
import os
from pathlib import Path

dataset_dir = Path('./IKEA-Manuals-at-Work')

if not dataset_dir.exists():
    print("Downloading IKEA dataset...")
    
    # Download from official source
    # Note: Update this URL with the actual dataset download link
    !wget -O ikea_dataset.zip "https://ikeamanuals.github.io/dataset/IKEA_Manuals_at_Work.zip"
    
    print("Extracting dataset...")
    !unzip -q ikea_dataset.zip
    !rm ikea_dataset.zip
    
    print("Dataset downloaded successfully!")
else:
    print("Dataset already exists.")

# Check dataset structure
!ls -lh IKEA-Manuals-at-Work/ | head -20

### Alternative: Use Google Drive

If direct download doesn't work, you can upload the dataset to Google Drive and mount it:

In [ ]:
# Uncomment if using Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Copy dataset from Drive (if you've uploaded it there)
# !cp -r /content/drive/MyDrive/IKEA-Manuals-at-Work .

## 3. Configure Training

In [ ]:
# Create GPU training configuration
config_content = """# IKEA MEPNet Training Configuration for Colab GPU

# Experiment settings
exp_name: "ikea_mepnet_colab"
exp_dir: "./experiments/ikea_mepnet_colab"
use_wandb: true  # Enable for experiment tracking
wandb_project: "ikea-mepnet"
use_tensorboard: true

# Model configuration
model:
  # Architecture
  num_stacks: 2
  num_blocks: 1
  num_features: 256
  image_size: [512, 512]

  # Shape conditioning
  use_shape_condition: true
  shape_encoding_dim: 512
  shape_encoder_type: "pointnet"

  # Output heads
  num_keypoints: 16
  num_rotation_bins: 24
  use_continuous_rotation: true
  max_parts: 10
  detect_parts: true
  predict_mask: true
  predict_depth: false

  # Training
  intermediate_supervision: true

# Data configuration
data:
  root_dir: "./IKEA-Manuals-at-Work"

  dataset_args:
    load_meshes: true  # Enable for full training
    load_videos: false
    furniture_categories: null  # null = all categories
    max_parts_per_step: 10
    image_size: [512, 512]
    normalize_scale: true
    cache_meshes: true

  augmentation:
    # Basic augmentations
    use_crop: true
    crop_size: [480, 480]
    preserve_center: true
    use_scale: true
    scale_range: [0.9, 1.1]
    use_color_jitter: true
    brightness: 0.2
    contrast: 0.2
    saturation: 0.2
    hue: 0.1

    # Manual-specific augmentations
    use_manual_style: true
    sketch_prob: 0.3
    edge_prob: 0.2
    binarize_prob: 0.1

    # Pose augmentation
    use_pose_noise: true
    rot_noise_deg: 3.0
    trans_noise_ratio: 0.03

    # Normalization
    normalize: true

# Loss configuration
loss:
  keypoint_weight: 1.0
  rotation_weight: 1.0
  translation_weight: 1.0
  mask_weight: 0.5
  part_detection_weight: 0.5
  depth_weight: 0.3
  use_continuous_rotation: true
  intermediate_supervision: true

# Optimization configuration
optimization:
  optimizer: "adamw"
  lr: 0.0001
  weight_decay: 0.01
  scheduler: "cosine"
  min_lr: 0.000001

# Training configuration
batch_size: 8  # Adjust based on GPU memory
val_batch_size: 8
epochs: 100  # Full training
num_workers: 2
device: "cuda"
use_multi_gpu: false
gradient_clip: 1.0

# Validation and saving
val_frequency: 1
save_frequency: 10

# Evaluation metrics
metrics:
  - "pose_accuracy"
  - "chamfer_distance"
  - "mask_iou"
  - "plan_accuracy"
  - "assembly_feasibility"

# Evaluation settings
save_predictions: true
"""

with open('configs/train_config_colab.yaml', 'w') as f:
    f.write(config_content)

print("Configuration saved to configs/train_config_colab.yaml")

## 4. Login to Weights & Biases (Optional)

W&B provides experiment tracking, visualization, and model versioning.

In [ ]:
# Login to W&B (get API key from https://wandb.ai/authorize)
import wandb

# Option 1: Interactive login
wandb.login()

# Option 2: Use API key directly
# wandb.login(key="YOUR_API_KEY_HERE")

## 5. Start Training

In [ ]:
# Train the model
!python scripts/train/train_ikea.py --config configs/train_config_colab.yaml

## 6. Monitor Training

Training progress can be monitored through:
- **W&B Dashboard**: Check your wandb project page
- **TensorBoard**: Run the cell below

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir experiments/ikea_mepnet_colab/tensorboard

## 7. Resume Training (if interrupted)

In [ ]:
# Find latest checkpoint
!ls -lht experiments/ikea_mepnet_colab/checkpoints/ | head -5

In [ ]:
# Resume from checkpoint
!python scripts/train/train_ikea.py \
    --config configs/train_config_colab.yaml \
    --resume experiments/ikea_mepnet_colab/checkpoints/checkpoint_latest.pth

## 8. Evaluate Model

In [ ]:
# Evaluate on test set
!python eval/eval_ikea.py \
    --config configs/train_config_colab.yaml \
    --checkpoint experiments/ikea_mepnet_colab/checkpoints/checkpoint_best.pth \
    --output_dir experiments/ikea_mepnet_colab/evaluation

## 9. Visualize Results

In [ ]:
# Visualize predictions
import matplotlib.pyplot as plt
from PIL import Image
import json
from pathlib import Path

# Load evaluation results
results_dir = Path('experiments/ikea_mepnet_colab/evaluation')

if results_dir.exists():
    # Show some prediction visualizations
    viz_dir = results_dir / 'visualizations'
    if viz_dir.exists():
        images = list(viz_dir.glob('*.png'))[:6]
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for idx, img_path in enumerate(images):
            img = Image.open(img_path)
            axes[idx].imshow(img)
            axes[idx].set_title(img_path.stem)
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Show metrics
    metrics_file = results_dir / 'metrics.json'
    if metrics_file.exists():
        with open(metrics_file) as f:
            metrics = json.load(f)
        
        print("\n=== Evaluation Metrics ===")
        for key, value in metrics.items():
            print(f"{key}: {value:.4f}")
else:
    print("No evaluation results found. Run evaluation first.")

## 10. Download Trained Model

In [ ]:
# Zip the experiment folder for download
!zip -r ikea_mepnet_trained.zip experiments/ikea_mepnet_colab/

# Download to local machine
from google.colab import files
files.download('ikea_mepnet_trained.zip')

## 11. Save to Google Drive (Recommended)

In [ ]:
# Copy results to Google Drive for safe keeping
from google.colab import drive
drive.mount('/content/drive')

!cp -r experiments/ikea_mepnet_colab /content/drive/MyDrive/
print("Results saved to Google Drive!")